In [2]:
!pip install pokerkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 1.3 MB/s eta 0:00:00


In [3]:
from math import inf
import numpy as np
import warnings
from tqdm import trange, tqdm
from copy import deepcopy
from math import inf
from pokerkit import *
import random
from concurrent.futures import ProcessPoolExecutor


warnings.filterwarnings("ignore", category=UserWarning)


In [4]:


#https://pokerkit.readthedocs.io/en/0.4/simulation.html

#Simulate a game given some information about the game.
class PokerSimulator:
    def __init__(self, number_of_player, seat=0):
        assert seat < number_of_player
        self.seat = seat #player position
        self.number_of_player = number_of_player

    def simulate_statecopy(self, state):
        self.state = deepcopy(state)
        #hands
        if len(self.state.hole_cards[self.seat]) == 0 :
            self.__deal_hands()
            self.__sample_betting()
            #every round there is betting
        #flop, turn, and river
        self.__sample_betting()

        board = self.state.board_cards
        if len(board) < 3 :
            self.__deal_sample_board()
            self.__sample_betting()

        if len(board) < 4 :
            self.__deal_sample_board()
            self.__sample_betting()

        #last round (river) betting:
        if len(board) < 5 :
            self.__deal_sample_board()
            self.__sample_betting()

        #show_down and payoffs
        self.__show_down()
        #Win or loses
        if self.state.payoffs[self.seat] >= 0:
            return 1
        else:
            return 0

    def simulate(self, myhands = None, _flop = None, _turn = None, _river = None):
        self.state = self.__make_samplegame(self.number_of_player) # game state object
        #Pre-flop: fill Holes
        # self.state.verify_hole_dealing() # (card, player index)

        #hands
        self.__deal_hands(myhands)
        #every round there is betting
        self.__sample_betting()
        #flop, turn, and river
        self.__deal_sample_board(_flop)
        self.__sample_betting()
        self.__deal_sample_board(_turn)
        self.__sample_betting()
        self.__deal_sample_board(_river)
        #last round (river) betting:
        self.__sample_betting()
        #show_down and payoffs
        self.__show_down()

        #Win or loses
        if self.state.payoffs[self.seat] >= 0:
            return 1
        else:
            return 0

    #make a game
    def __make_samplegame(self, number_of_player):
        #Creating game template
        sample_game = NoLimitTexasHoldem(
        #positions : zeroth player will always be the small blind,
            #the first player will always be the big blind
            (
                Automation.ANTE_POSTING,
                Automation.BET_COLLECTION,
                Automation.BLIND_OR_STRADDLE_POSTING,
                Automation.HAND_KILLING,
                Automation.CHIPS_PUSHING,
                Automation.CHIPS_PULLING,
            ),
            True,  # Uniform antes? (False for big blind ante)
            1,     # antes
            (0,0),  # Blinds or straddles
            1,      # Min-bet
        )
        return sample_game(
            inf, #starting stacks
            number_of_player,
        )

    def __sample_betting(self):
        while self.state.can_check_or_call():
            self.state.check_or_call()

    def __deal_hands(self, hands = None):
        #hands
        for i in range(self.number_of_player):
            if i == self.seat and hands != None:
                self.state.deal_hole(hands) #deal my cards
            else:
                self.state.deal_hole("????") #deal opp hands: two random cards

    def __deal_sample_board(self, cards = None):
        if(self.state.can_burn_card()):
            self.state.burn_card("??") #burn a random card which not affect what we can deal next

        while(self.state.can_deal_board()):
            if cards :
                self.state.deal_board(cards) #initialize flop
            else:
                self.state.deal_board() #randomly deal board

    def __show_down(self):
        # for i in range(5):
        #     if self.state.board_cards[i][0].unknown_status:
        #         self.state.board_cards[i][0] = self.state.deck_cards.popleft()

        while self.state.showdown_index != None:
            i = self.state.showdown_index
            hand1, hand2 = self.state.hole_cards[i]
            if (hand1.unknown_status == True) : #if the hand is [??, ??]
                hand1 = self.state.deck_cards.popleft() #draw two cards from the deck
                hand2 = self.state.deck_cards.popleft()
            self.state.show_or_muck_hole_cards((hand1,hand2), player_index= i) #reveal

In [5]:


#Resample Methods

def resample(players: list[Card], board: list[Card], dealable_cards: list[Card],
            non_evidence_player_indices: list[int],
            non_evidence_board_indices: list[int]):
    def resample_player(p: list[Card], dealable_cards: list[Card]):
        dealable_cards.append(p[0])
        dealable_cards.append(p[1])
        p[0] = random.choice(dealable_cards)
        dealable_cards.remove(p[0])
        p[1] = random.choice(dealable_cards)
        dealable_cards.remove(p[1])

    def resample_board_cards(board: list[Card], board_index: int, dealable_cards: list[Card]):
        dealable_cards.append(board[board_index])
        board[board_index] = random.choice(dealable_cards)
        dealable_cards.remove(board[board_index])

    if(len(board) == 0):
        player_index = random.choice(non_evidence_player_indices)
        resample_player(players[player_index], dealable_cards)
    else:
        choice = random.choice(range(2))
        if choice:
            resample_board_cards(board, random.choice(non_evidence_board_indices), dealable_cards)
        else:
            player_index = random.choice(non_evidence_player_indices)
            resample_player(players[player_index], dealable_cards)

#-________________________________________________-|- ^ -|
def MCMC(number_of_player: int,
        seat: int,
        state: State,
        number_of_iteration: int = 1000):
    #======Internal Logic=====
    def initialize_values(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices):
        for i in non_evidence_player_indices:
            players[i][0]= random.choice(dealable_cards)
            dealable_cards.remove(players[i][0])
            players[i][1]= random.choice(dealable_cards)
            dealable_cards.remove(players[i][1])
        # Initialize the board
        for j in non_evidence_board_indices:
            board.append(random.choice(dealable_cards))
            dealable_cards.remove(board[j])

    def get_non_evidence_indices(number_of_player, players, board):
        non_evidence_player_indices = []
        for i in range(number_of_player):
            if players[i][0].unknown_status:
                non_evidence_player_indices.append(i)
        non_evidence_board_indices = [i for i in range(len(board), 5)]
        return non_evidence_player_indices,non_evidence_board_indices

    _state = deepcopy(state)
    players, board = deepcopy(_state.hole_cards), deepcopy(_state.board_cards)
    for i in range(len(board)):
        board[i] = board[i][0]

    dealable_cards = [card for card in _state.get_dealable_cards()]

    non_evidence_player_indices, non_evidence_board_indices = get_non_evidence_indices(number_of_player, players, board)
    initialize_values(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices)
    count_win = 0

    for _ in range(number_of_iteration):
        resample(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices)
        #Evaluation
        my_hand = StandardHighHand.from_game(players[seat], board)
        opp_hand = [StandardHighHand.from_game(players[i], board) for i in range(number_of_player) if i != seat]
        win = all(my_hand > opp_hand[i] for i in range(number_of_player-1))
        count_win += 1 if win else 0

    return count_win / number_of_iteration


In [25]:

#Monte Carlo
def PW_E_Monte_Carlo(numb_player : int, seat : int, state : State):
  simulator = PokerSimulator(number_of_player=numb_player, seat = seat)

  num_of_games = 200 #We choose 200
  count = 0
  for i in (range(num_of_games)):
    count += simulator.simulate_statecopy(state)
  return count / num_of_games

#Monte Carlo Markov Chain

def MCMC_agent(numb_player, seat, state):
  cpt = MCMC(numb_player, seat, state)
  return Action(cpt, state)

#Agent
def Action(win_probability, state : State):
  game_stage = len(state.board_cards)
  if game_stage == 0: #After dealin hands
      if win_probability > 0.10:
        return "CHECK"
      else:
        return "FOLD"
  if game_stage == 3: #Flop
      if win_probability > 0.20:
        return "CHECK"
      else:
        return "FOLD"
  if game_stage == 4: #Turn
      if win_probability > 0.20:
        return "CHECK"
      else:
        return "FOLD"
  if game_stage == 5: #River
      if win_probability > 0.30:
        return "CHECK"
      else:
        return "FOLD"

  return "WHAT"


#Agent2 Still a simple action table
def MC_agent2(numb_player: int, seat : int, state):
  cpt = PW_E_Monte_Carlo(numb_player, seat, state)
  return Action(cpt, state)


#=======Old Version Bin=======
def MC_agent1(number_player : int, seat : int, state : State):
  if PW_E_Monte_Carlo(number_player, seat, state) > 0.15:
    return "CHECK"
  return "FOLD"


In [ ]:

#Game
class PokerGame:
    def __init__(self, number_of_player =6, seat = 1, agent= None, show_log = False, seed = None):
        assert seat < number_of_player
        if seed != None:
            random.seed(seed)
        self.seat = seat
        self.num_of_players = number_of_player
        self.player_to_act = False
        self.show_log = show_log
        self.state = self.__make_game(number_of_player)
        self.evaluation_metrics = {"TC": 0, "FC": 0, "TF": 0, "FF": 0}

    def __make_game(self, number_of_player):
        #Creating game template
        self.minBet = 1
        self.sample_game = NoLimitTexasHoldem(
        #positions : zeroth player will always be the small blind,
        #the first player will always be the big blind
            (
                Automation.ANTE_POSTING,
                Automation.BET_COLLECTION,
                Automation.BLIND_OR_STRADDLE_POSTING,
                Automation.HAND_KILLING,
                Automation.CHIPS_PUSHING,
                Automation.CHIPS_PULLING,
            ),
            True,   # Uniform antes? (False for big blind ante)
            1,      # antes (The minimum bet every play contribute at the start of the game)
            (0,0),  # Blinds or straddles
            self.minBet,      # Min-bet
        )
        return self.sample_game(
            inf, #starting stacks
            number_of_player,
        )

    '''
    ******************************
            get() FUNctions
    ******************************
    '''
#Payoffs:
    def payoffs(self):
        return self.state.payoffs[self.seat]
#Evaluation Metric Counter
    def get_TC_count(self):
        return self.evaluation_metrics['TC']

    def get_TF_count(self):
        return self.evaluation_metrics['TF']

    def get_FC_count(self):
        return self.evaluation_metrics['FC']

    def get_FF_count(self):
        return self.evaluation_metrics['FF']

#Cards:
    def get_board_status(self):
        return self.get_Hands(), self.get_Flop(), self.get_Turn(), self.get_River()

    def get_Hands(self):
        return self.state.hole_cards[self.seat] if (len(self.state.hole_cards[self.seat]) == 2 )else None

    def get_Flop(self):
        board = self.state.board_cards
        if len(board) < 3:
            return None
        flop = board[0:3]
        return [flop[0][0], flop[1][0], flop[2][0]]

    def get_Turn(self):
        board = self.state.board_cards
        if len(board) < 4:
            return None
        return board[3]

    def get_River(self):
        board = self.state.board_cards
        if len(board) < 5:
            return None
        return board[4]


    def start_round(self, agent_function = None, hands = None, flop = None, turn = None, river = None):
        self.state = self.sample_game( # Reset the Game State
            inf, #starting stacks
            self.num_of_players,
        )
        self.Fold = False
        self.deal_board_state = 0
        if self.show_log:
            print(f"Your Seat is {self.seat}")

        # Preflop
        self.__deal_hands(hands)
        if self.show_log:
            print(f"Your hands are {self.state.hole_cards[self.seat]}")
        self.__betting()
        self.__player_action(agent_function)

        # Postflop, Turn, River
        board_cards = [flop, turn, river]
        while self.state.showdown_index is None:  # Until showdown
            if self.show_log:
                print("Dealt Card(s)")
            self.__deal_board(board_cards)
            self.deal_board_state += 1
            if self.show_log:
                print(f"Board: {self.state.board_cards}")
            self.__betting()
            self.__player_action(agent_function)

        # Showdown
        self.__show_down()

        # Evaluate Outcome
        result = self.__evaluation()
        if self.show_log:
            print(f"Player {np.argmax(self.state.payoffs)} Wins")
            print(f"Result: {result}, Your Payoff: {self.payoffs()}")

        return result

#Private Functions.
    def __evaluation(self):
        actual_win = self.payoffs() > 0  # True if player actually won
        predicted_win = not self.Fold  # True if player did NOT fold
        if actual_win and predicted_win:
            result = "TC"  # True Positive
        elif not actual_win and predicted_win:
            result = "FC"  # False Positive
        elif not actual_win and not predicted_win:
            result = "TF"  # True Negative
        else:  # actual_win and not predicted_win
            result = "FF"  # False Negative

        self.evaluation_metrics[result] += 1
        return result
    def __player_action(self, agent_function):
        assert self.player_to_act

        if self.Fold: # Plater already fold, no action allow
            self.state.check_or_call()
            self.player_to_act = False
            self.__betting() #Pass it to next player
            return

        hand, flop, river, turn = self.get_board_status()
        while self.player_to_act:
            '''=====PLAYER ACION====='''
            action_list= ['Check', 'Call', 'Bet', 'Look', 'Fold']
            if agent_function != None:
                action = agent_function(self.num_of_players, self.seat, self.state)
                # action = agent_function(self.num_of_players, self.seat, hand, flop, river, turn)
            else:
                #default
                action = random.choice(action_list)
                action = action.upper().strip().strip('\n')

            '''===Action Logistics==='''
            if action == "LOOK":
                if self.show_log:
                    print(f"Board : {self.state.board_cards}\n \
                Your Hands: {self.state.hole_cards[self.seat]}")
            elif action == "FOLD":
                if self.show_log:
                    print("You Fold")

                """Save the Bet amont
                and Other info to save"""
                self.Fold = True

                self.state.check_or_call()
                self.player_to_act = False
                self.__betting() #Pass it to next player
                #We need to evaluate our hand at the end
            elif action == "CHECK" or action == "CALL":
                self.state.check_or_call()
                self.player_to_act = False
                self.__betting() #Pass it to next player
            elif action == "BET":
                amount = 90
                self.bet_or_raise_to(amount)
                self.player_to_act = False
                self.__betting() #Pass it to next player
            #End of player action

    def __betting(self):
        while self.state.can_check_or_call():
            if self.state.actor_index == self.seat:
                self.player_to_act = True
                return
            #opp always check or call
            self.state.check_or_call()

    def __deal_hands(self,hands):
        while self.state.can_deal_hole():
            if self.state.hole_dealee_index == self.seat:
                if(hands != None):
                    self.state.deal_hole(hands)
                else:
                    self.state.deal_hole() #deal my cards
            else:
                self.state.deal_hole() #deal opp hands: two random cards

    def __deal_board(self, given_boards):
        assert self.state.can_burn_card()
        self.state.burn_card("??") #burn a random card which not affect what we can deal next
        while(self.state.can_deal_board()):
            if given_boards[self.deal_board_state] != None:
                self.state.deal_board(given_boards[self.deal_board_state])
            else:
                self.state.deal_board() #randomly deal board

    def __show_down(self):
        #If the board_cards is unknown ??
        if self.show_log :
            print("====Show Down====")
        for i in range(len(self.state.board_cards)):
            if self.state.board_cards[i][0].unknown_status:
                self.state.board_cards[i][0] = self.state.deck_cards.popleft()
        while self.state.showdown_index != None:
            i = self.state.showdown_index
            hand1, hand2 = self.state.hole_cards[i]
            if hand1.unknown_status == True : #if the hand is [??, ??]
                hand1 = self.state.deck_cards.popleft() #draw two cards from the deck
                hand2 = self.state.deck_cards.popleft()
            if self.show_log :
                if i == self.seat:
                    print("(You) ", end = "")
                print(f"Player {i} hands : ({hand1,hand2})")
            self.state.show_or_muck_hole_cards((hand1,hand2), player_index= i) #reveal

In [14]:
def resample(players: list[Card], board: list[Card], dealable_cards: list[Card],
            non_evidence_player_indices: list[int],
            non_evidence_board_indices: list[int]):
    def resample_player(p: list[Card], dealable_cards: list[Card]):
        dealable_cards.append(p[0])
        dealable_cards.append(p[1])
        p[0] = random.choice(dealable_cards)
        dealable_cards.remove(p[0])
        p[1] = random.choice(dealable_cards)
        dealable_cards.remove(p[1])

    def resample_board_cards(board: list[Card], board_index: int, dealable_cards: list[Card]):
        dealable_cards.append(board[board_index])
        board[board_index] = random.choice(dealable_cards)
        dealable_cards.remove(board[board_index])



    choice = random.choice(range(2))  # Randomly choose between 0 and 1

    if choice:  # If choice is 1
        # Resample board if not empty, or if full and non_evidence_board_indices is not empty
        if board or non_evidence_board_indices:
            resample_board_cards(board, random.choice(non_evidence_board_indices or range(len(board))), dealable_cards)
    else:  # If choice is 0
        # Resample player hands
        player_index = random.choice(non_evidence_player_indices)
        resample_player(players[player_index], dealable_cards)
#    if(len(board) == 0):
#        player_index = random.choice(non_evidence_player_indices)
#        resample_player(players[player_index], dealable_cards)
#    else:
#        choice = random.choice(range(2))
#        if choice:
#            resample_board_cards(board, random.choice(non_evidence_board_indices), dealable_cards)
#        else:
#            player_index = random.choice(non_evidence_player_indices)
#            resample_player(players[player_index], dealable_cards)

#-________________________________________________-|- ^ -|
def MCMC(number_of_player: int,
        seat: int,
        state: State,
        number_of_iteration: int = 1000):
    #======Internal Logic=====
    def initialize_values(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices):
        for i in non_evidence_player_indices:
            players[i][0]= random.choice(dealable_cards)
            dealable_cards.remove(players[i][0])
            players[i][1]= random.choice(dealable_cards)
            dealable_cards.remove(players[i][1])
        # Initialize the board
        for j in non_evidence_board_indices:
            board.append(random.choice(dealable_cards))
            dealable_cards.remove(board[j])

    def get_non_evidence_indices(number_of_player, players, board):
        non_evidence_player_indices = []
        for i in range(number_of_player):
            if players[i][0].unknown_status:
                non_evidence_player_indices.append(i)
        non_evidence_board_indices = [i for i in range(len(board), 5)]
        return non_evidence_player_indices,non_evidence_board_indices

    _state = deepcopy(state)
    players, board = deepcopy(_state.hole_cards), deepcopy(_state.board_cards)
    for i in range(len(board)):
        board[i] = board[i][0]

    dealable_cards = [card for card in _state.get_dealable_cards()]

    non_evidence_player_indices, non_evidence_board_indices = get_non_evidence_indices(number_of_player, players, board)
    initialize_values(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices)
    count_win = 0

    for _ in range(number_of_iteration):
        resample(players, board, dealable_cards, non_evidence_player_indices, non_evidence_board_indices)
        #Evaluation
        my_hand = StandardHighHand.from_game(players[seat], board)
        opp_hand = [StandardHighHand.from_game(players[i], board) for i in range(number_of_player) if i != seat]
        win = all(my_hand > opp_hand[i] for i in range(number_of_player-1))
        count_win += 1 if win else 0

    return count_win / number_of_iteration


In [9]:
number_of_player = 6
minBet = 1
sample_game = NoLimitTexasHoldem(
#positions : zeroth player will always be the small blind,
#the first player will always be the big blind
    (
        Automation.ANTE_POSTING,
        Automation.BET_COLLECTION,
        Automation.BLIND_OR_STRADDLE_POSTING,
        Automation.HAND_KILLING,
        Automation.CHIPS_PUSHING,
        Automation.CHIPS_PULLING,
    ),
    True,   # Uniform antes? (False for big blind ante)
    1,      # antes (The minimum bet every play contribute at the start of the game)
    (0,0),  # Blinds or straddles
    minBet,      # Min-bet
)
state = sample_game(
    inf, #starting stacks
    number_of_player,
)

seat = 2
#Every Player has a hand
for _ in range(number_of_player):
    if _ == seat :
        state.deal_hole("AcAd")
    else:
        state.deal_hole("????")
print(state.hole_cards)

print(MCMC(number_of_player,seat, state))

[[??, ??], [??, ??], [Ac, Ad], [??, ??], [??, ??], [??, ??]]
0.521


In [27]:
num_games = 1000 # Number of games to simulate per agent


def run_games(pk, agent, num_games, description):
    results = []
    for i in trange(num_games, desc=description):
        results.append(pk.start_round(agent))
    return results


pk = PokerGame(show_log=False, seed=6969)

with ProcessPoolExecutor() as executor:
    futures = {
        "mcmc": executor.submit(run_games, pk, MCMC_agent, num_games, "Simulating MCMC Agent Games"),
        "mc": executor.submit(run_games, pk, MC_agent1, num_games, "Simulating MC Agent Games")
    }

    # Get the results and process them
    for agent_name, future in futures.items():
        results = future.result()  # Get the list of game results

        true_pos, false_pos, true_neg, false_neg = 0, 0, 0, 0
        for game in results:
            if game == "TC":
                true_pos += 1
            elif game == "FC":
                false_pos += 1
            elif game == "TF":
                true_neg += 1
            elif game == "FF":
                false_neg += 1

        print(f"\nWinning Accuracy for {agent_name.upper()} Agent (out of {num_games} games):")
        print(f"True Positive: {true_pos}")
        print(f"False Positive: {false_pos}")
        print(f"True Negative: {true_neg}")
        print(f"False Negative: {false_neg}")
        print(f"Winning Accuracy: {true_pos / (true_pos + false_neg):.4f}")
        print(f"Losing Accuracy: {true_neg / (true_neg + false_pos):.4f}")
        if true_pos +false_pos != 0:
          print(f"P(Win|Check): {true_pos / (true_pos + false_pos):.4f}")
        if true_neg + false_neg != 0:
          print(f"P(Lose|Fold): {true_neg / (true_neg + false_neg):.4f}")

Simulating MCMC Agent Games: 100%|██████████| 1000/1000 [51:09<00:00,  3.07s/it]



Winning Accuracy for MCMC Agent (out of 1000 games):
True Positive: 8
False Positive: 13
True Negative: 796
False Negative: 183
Winning Accuracy: 0.0419
Losing Accuracy: 0.9839
P(Win|Check): 0.3810
P(Lose|Fold): 0.8131


Simulating MC Agent Games: 100%|██████████| 1000/1000 [57:25<00:00,  3.45s/it]



Winning Accuracy for MC Agent (out of 1000 games):
True Positive: 164
False Positive: 5
True Negative: 831
False Negative: 0
Winning Accuracy: 1.0000
Losing Accuracy: 0.9940
P(Win|Check): 0.9704
P(Lose|Fold): 1.0000
